In [52]:
import tensorflow as tf
import pandas as pd
import numpy as np
from keras.preprocessing.text import Tokenizer
from keras.layers import Bidirectional, Dense, Embedding, LSTM
from keras.utils import pad_sequences

In [53]:
def clean(data: pd.DataFrame):
    data["train"] = data[data["train"] != ""]
    data.dropna(subset=["train"], inplace=True)

def getLines(song: str):
    return song.split("\n")


In [54]:
FILEPATH = "./etc/imagine_dragons_lyrics.json"
data = pd.read_json(FILEPATH) #["train"]

# Remove empty entries
clean(data)

# Split each song into the constituent lines
data["lyrics"] = data["train"].apply(getLines)

In [55]:
corpus = [item for sublist in data["lyrics"].values for item in sublist]

# Tokenize the corpus
tokenizer = Tokenizer()
tokenizer.fit_on_texts(corpus)

# We add 1 for the token used to pad sequences
total_words = len(tokenizer.word_index) + 1

print(tokenizer.word_index)
print("Total words: ", total_words)

# This will be used to pad the other sequences
max_sequence_len = 0

# Generate input sequences of ngrams to feed to the LSTM model
input_sequences = []
for line in corpus:
    line_tokens = tokenizer.texts_to_sequences([line])[0]
    tokens_len = len(line_tokens)
    max_sequence_len = max(max_sequence_len, tokens_len)

    # Create ngrams for feeding into LSTM
    # Starts from 1 because otherwise we'll get an empty list as the first element
    for token in range(1, tokens_len):
        n_gram_sequence = line_tokens[:token+1]
        input_sequences.append(n_gram_sequence)

print(max_sequence_len)

# Pad the sequences
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding="pre"))

features, labels = input_sequences[:, :-1], input_sequences[:,-1]

# One-hot encode the labels
y = tf.keras.utils.to_categorical(labels, num_classes=total_words)

print(features)

{'i': 1, 'the': 2, 'you': 3, 'to': 4, 'a': 5, 'my': 6, 'and': 7, 'me': 8, 'oh': 9, 'im': 10, 'it': 11, 'of': 12, 'that': 13, 'in': 14, 'all': 15, 'we': 16, 'its': 17, 'on': 18, 'for': 19, 'dont': 20, 'this': 21, 'your': 22, 'up': 23, 'is': 24, 'just': 25, 'down': 26, 'be': 27, 'la': 28, 'can': 29, 'but': 30, 'let': 31, 'life': 32, 'know': 33, 'when': 34, 'never': 35, 'love': 36, 'so': 37, 'time': 38, 'with': 39, 'now': 40, 'what': 41, 'out': 42, 'am': 43, 'ooh': 44, 'are': 45, 'thunder': 46, 'was': 47, 'one': 48, 'do': 49, 'go': 50, 'get': 51, 'ill': 52, 'ive': 53, 'bet': 54, 'no': 55, 'were': 56, 'cause': 57, 'at': 58, 'everything': 59, 'take': 60, 'been': 61, 'wanna': 62, 'come': 63, 'got': 64, 'want': 65, 'from': 66, 'feel': 67, 'hey': 68, 'youre': 69, 'they': 70, 'not': 71, 'back': 72, 'shot': 73, 'whoa': 74, 'then': 75, 'where': 76, 'if': 77, 'da': 78, 'like': 79, 'who': 80, 'away': 81, 'night': 82, 'over': 83, 'tell': 84, 'see': 85, 'through': 86, 'give': 87, 'make': 88, 'every':

In [59]:
EMBEDDING_DIMS = 256

# Construct the model
model = tf.keras.Sequential([
    Embedding(input_dim=total_words, output_dim=EMBEDDING_DIMS, input_length=max_sequence_len-1),
    Bidirectional(LSTM(256, return_sequences=True)), # stateful=True,
    Bidirectional(LSTM(128)), # stateful=True,
    Dense(128, activation="relu"),
    Dense(total_words, activation="softmax")
])
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01),
              loss="categorical_crossentropy",
              metrics=["accuracy"])
model.summary()

Model: "sequential_9"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 31, 256)           860416    
                                                                 
 bidirectional_20 (Bidirecti  (None, 31, 512)          1050624   
 onal)                                                           
                                                                 
 bidirectional_21 (Bidirecti  (None, 256)              656384    
 onal)                                                           
                                                                 
 dense_10 (Dense)            (None, 128)               32896     
                                                                 
 dense_11 (Dense)            (None, 3361)              433569    
                                                                 
Total params: 3,033,889
Trainable params: 3,033,889
No

In [61]:
EPOCHS = 100
history = model.fit(features, y, epochs=EPOCHS, verbose=1)

Epoch 1/100
  15/1876 [..............................] - ETA: 14:07 - loss: 6.0964 - accuracy: 0.0271

KeyboardInterrupt: 